# Tools for friction law development
Last modification: 2024.01.30.</br>

# Initiate

In [ ]:
import numpy as np
import setup_shortcut
import change_params
from read_outputs import *
import matplotlib.pylab as plt
from scipy import signal,interpolate
import myplots
mp = myplots.Figpref()
plt.rcParams['font.size'] = '15'

sc = setup_shortcut.setups()
ch = change_params.variate()

target_eventid = 8
save_dir = 'source_test/seissol_outputs'
model_n = 'vert_slow'
mu = 0.4
receivef_strike = 340

save_dir1 = '/raid/class239/jeena/Tandem/'+'source_test/base_pert'
save_dir2 = '/raid/class239/jeena/Tandem/'+'source_test/original_source'
save_dir3 = '/raid/class239/jeena/Tandem/'+'source_test/custom_source'

# outputs1,dep1,params1 = load_short_fault_probe_outputs(save_dir1,0)
if 'save_dir1' in locals(): outputs,dep = load_fault_probe_outputs(save_dir1)
if 'save_dir2' in locals(): outputs2,dep2 = load_fault_probe_outputs(save_dir2)
if 'save_dir3' in locals(): outputs3,dep3 = load_fault_probe_outputs(save_dir3)

# Check state evolution

In [ ]:
target_depth = 7.5
di = np.argmin(abs(abs(dep)-target_depth))
time,psi,sigma,sliprate = outputs[di,:,0],outputs[di,:,1],outputs[di,:,5]*1e6,outputs[di,:,4]
time2,psi2,sigma2,sliprate2 = outputs2[di,:,0],outputs2[di,:,1],outputs2[di,:,5]*1e6,outputs2[di,:,4]
time3,psi3,sigma3,sliprate3 = outputs3[di,:,0],outputs3[di,:,1],outputs3[di,:,5]*1e6,outputs3[di,:,4]

In [ ]:
plt.figure(figsize=(8,6))
gap = 1e5
# ii = np.where(np.logical_and(time>=min([time2[0],time3[0]])-gap,time<=max([time2[-1],time3[-1]])+gap))
# plt.plot(time[ii],psi[ii],'k',lw=2.5,label='Base (Aging Law)')
plt.plot(time,psi,'k',lw=2.5,label='Base (Aging Law)')
plt.plot(time2,psi2,c=mp.myburgundy,lw=2.5,label='source(x,y,t)',linestyle='--')
plt.plot(time3,psi3,c=mp.myyellow,lw=2.5,label='source(x,y,t,sn)',linestyle='--')
plt.legend(fontsize=15)
plt.xlabel('Time [s]',fontsize=17)
plt.ylabel('$\psi$',fontsize=17)
plt.title('%1.1f km'%(abs(dep[di])),fontsize=17,fontweight='bold')
gap = 0.25
plt.xlim(min([time2[0],time3[0]])-gap,max([time2[-1],time3[-1]])+gap)
plt.show()

In [ ]:
def get_lhs_rhs(time,psi,sigma,V,alpha=0.3):
    b = 0.019
    V0 = 1e-6
    L = 0.008
    f0 = 0.6
    dpsidt = np.gradient(psi,time,edge_order=2)
    dsigmadt = np.gradient(sigma,time,edge_order=2)
    # dpsidt = np.hstack((0,np.diff(psi)/np.diff(time)))
    # dsigmadt = np.hstack((0,np.diff(sigma)/np.diff(time)))
    first_term = (b*V0/L) * (np.exp((f0 - psi)/b) - abs(V)/V0)
    rhs = first_term - alpha*dsigmadt/sigma
    lhs = dpsidt
    return lhs,rhs

lhs,rhs = get_lhs_rhs(time,psi,sigma,sliprate,alpha=0)
lhs2,rhs2 = get_lhs_rhs(time2[1:],psi2[1:],sigma2[1:],sliprate2[1:],alpha=0.3)
lhs3,rhs3 = get_lhs_rhs(time3,psi3,sigma3,sliprate3,alpha=0.3)

In [ ]:
plt.figure(figsize=(12,6))
plt.plot(time2[1:]-time2[1],lhs2,c=mp.myburgundy,lw=2,label='LHS',linestyle='--',zorder=3)
plt.plot(time2[1:]-time2[1],rhs2,'k',lw=2,label='RHS')
# plt.plot(time2-time2[0],lhs2,c=mp.myburgundy,lw=2,label='LHS',linestyle='--',zorder=3)
# plt.plot(time2-time2[0],rhs2,'k',lw=2,label='RHS')
# plt.scatter(time2-time2[0],lhs2,fc=mp.myburgundy,label='LHS',zorder=3)
# plt.scatter(time2-time2[0],rhs2,fc='k',label='RHS')
plt.legend(fontsize=15)
plt.xlabel('Time [s]',fontsize=17)
plt.ylabel('d$\psi$/dt',fontsize=17)
plt.title('source(x,y,t); %1.1f km'%(abs(dep[di])),fontsize=17,fontweight='bold')
plt.xlim(0,5)
plt.show()

In [ ]:
plt.figure(figsize=(12,6))
plt.scatter(time2[1:]-time2[1],sliprate2[1:],fc=mp.myburgundy,zorder=3)
# plt.scatter(time2-time2[0],rhs2,fc='k',label='RHS')
# plt.legend(fontsize=15)
plt.xlabel('Time [s]',fontsize=17)
plt.ylabel('$\sigma$',fontsize=17)
plt.title('source(x,y,t); %1.1f km'%(abs(dep[di])),fontsize=17,fontweight='bold')
plt.xlim(0,1)
plt.show()

# Check other variables

In [ ]:
di = np.argmin(abs(abs(dep)-7.5))
# vari,ylab = 1, '$\psi$' # state
# vari,ylab = 2, 'Slip [m]' # slip
# vari,ylab = 3,'Shear Stress [MPa]' # shearT
# vari,ylab = 4,'$log_{10}$(Slip Rate [m/s])' # sliprate
vari,ylab = 5,'Normal Stress [MPa]' # normal stress
if vari == 4:
    time,var = outputs[di,:,0],np.log10(abs(outputs[di,:,vari]))
    time2,var2 = outputs2[di,:,0],np.log10(outputs2[di,:,vari])
    time3,var3 = outputs3[di,:,0],np.log10(outputs3[di,:,vari])
else:
    time,var = outputs[di,:,0],outputs[di,:,vari]
    time2,var2 = outputs2[di,:,0],outputs2[di,:,vari]
    time3,var3 = outputs3[di,:,0],outputs3[di,:,vari]

In [ ]:
plt.figure(figsize=(8,6))
gap = 1e5
# ii = np.where(np.logical_and(time>=min([time2[0],time3[0]])-gap,time<=max([time2[-1],time3[-1]])+gap))
# plt.plot(time[ii],var[ii],'k',lw=2.5,label='Base (Aging Law)')
plt.plot(time,var,'k',lw=2.5,label='Base (Aging Law)')
plt.plot(time2,var2,c=mp.myburgundy,lw=2.5,label='source(x,y,t)',linestyle='--')
plt.plot(time3,var3,c=mp.myyellow,lw=2.5,label='source(x,y,t,sn)',linestyle='--')
plt.legend(fontsize=15)
plt.xlabel('Time [s]',fontsize=17)
plt.ylabel(ylab,fontsize=17)
plt.title('%1.1f km'%(abs(dep[di])),fontsize=17,fontweight='bold')
gap = 2
plt.xlim(min([time2[0],time3[0]])-gap,max([time2[-1],time3[-1]])+gap)
plt.show()

In [ ]:
delPn = -np.loadtxt('%s/ssaf_%s_Pn_pert_mu%02d_%d.dat'%(save_dir,model_n,int(mu*10),receivef_strike))
depth_range = np.loadtxt('%s/ssaf_%s_dep_stress_pert_mu%02d_%d.dat'%(save_dir,model_n,int(mu*10),receivef_strike))
dt = 0.01

dri = np.argmin(abs(abs(depth_range)-7.5))
sn = 50 + delPn[:,dri]
dsndt = np.hstack((0,np.diff(delPn[:,dri])/dt))